In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import qutip as qt

# Hamiltonians

## General Alice Bob Cavity Hamiltonian

Let the Hamiltonian for Alice (A), Bob (B), and a coupler mode (C), all modeled as cavity modes with annihilation operators $a$, $b$, and $c$ and frequencies $\omega_A$, $\omega_B$, $\omega_C$, be:

$$
H = \omega_A\, a^\dagger a + \omega_B\, b^\dagger b + \omega_C\, c^\dagger c  + g_{AC} (a^\dagger c + a c^\dagger) + g_{BC} (b^\dagger c + b c^\dagger)
$$

where:
- $a$, $b$, $c$ are annihilation operators for Alice, Bob, and the coupler mode,
- $a^\dagger$, $b^\dagger$, $c^\dagger$ are the corresponding creation operators,
- $g_{AC}$, $g_{BC}$ are couplings between Alice/Coupler and Bob/Coupler.

In [5]:
# Parameters for the general Alice-Bob-Cavity Hamiltonian (all units are in MHz)
omega_A = 2 * np.pi * 5e3      # Frequency of Alice's mode
omega_B = 2 * np.pi * 6e3      # Frequency of Bob's mode
omega_C = 2 * np.pi * 4e3      # Frequency of Coupler mode
g_AC = 2 * np.pi * 100         # Coupling between Alice and Coupler
g_BC = 2 * np.pi * 100         # Coupling between Bob and Coupler


In [6]:
# Define truncation for each mode
trunc = 5

# Creation and annihilation operators for Alice (a), Bob (b), and Coupler (c)
a = qt.destroy(trunc)      # Alice annihilation
a_dag = a.dag()            # Alice creation

b = qt.destroy(trunc)      # Bob annihilation
b_dag = b.dag()            # Bob creation

c = qt.destroy(trunc)      # Coupler annihilation
c_dag = c.dag()            # Coupler creation

In [7]:
def generate_alice_bob_cavity_hamiltonian(omega_A, omega_B, omega_C, g_AC, g_BC, a, a_dag, b, b_dag, c, c_dag):
    """
    Generate the Hamiltonian:
        H = omega_A * a†a + omega_B * b†b + omega_C * c†c
            + g_AC * (a†c + a c†) + g_BC * (b†c + b c†)
    """
    H = (
        omega_A * a_dag * a
        + omega_B * b_dag * b
        + omega_C * c_dag * c
        + g_AC * (a_dag * c + a * c_dag)
        + g_BC * (b_dag * c + b * c_dag)
    )
    return H


## LINC Potential 
$$U = 2M^2E_J \cos{\phi_{ext}}\cos{\theta/M}$$
- $M$: Integer parameter related to the number of junctions or modes in the system  
- $E_J$: Josephson energy  
- $\phi_{ext}$: External flux phase  
- $\theta$: Collective phase variable, e.g., $\theta = \theta_{\mathrm{zpt}} (c + c^\dagger)$  
- $\theta_{\mathrm{zpt}} = (2E_C/E_L)^{1/4}$
- $U$: LINC potential energy

If we set $\phi_{ext} = \frac{\pi}{2} + \phi_{AC}$, then H becomes:
$$
U = 2M^2E_J \sin{\phi_{AC}}\cos{\theta/M}
$$



In [ ]:
# Parameters for LINC potential (example values, adjust as needed)
M = 3                # Integer parameter (number of junctions/modes)
E_J = 2 * np.pi * 15e3    # Josephson energy (in appropriate units)
# phi_AC = np.pi / 4   # Phase variable (in radians)
E_C = 2 * np.pi * 100e3   # Charging energy
E_L = 2 * np.pi * 52.8    # Inductive energy


In [8]:
def theta_operator(E_C, E_L, c, c_dag):
    """
    Construct the theta operator:
        theta = theta_zpt * (c + c†)
        where theta_zpt = (2 * E_C / E_L) ** 0.25

    Parameters
    ----------
    E_C : float
        Charging energy.
    E_L : float
        Inductive energy.
    c : Qobj
        Coupler annihilation operator.
    c_dag : Qobj
        Coupler creation operator.

    Returns
    -------
    theta : Qobj
        The theta operator.
    """
    theta_zpt = (2 * E_C / E_L) ** 0.25
    return theta_zpt * (c + c_dag)

def linc_potential_operator(M, E_J, phi_AC, E_C, E_L, c, c_dag):
    """
    Generate the LINC potential energy operator:
        U = 2 * M**2 * E_J * sin(phi_AC) * cos(theta / M)
    where theta = theta_zpt * (c + c†), theta_zpt = (2 * E_C / E_L) ** 0.25

    Parameters
    ----------
    M : int
        Integer parameter related to the number of junctions or modes.
    E_J : float
        Josephson energy.
    phi_AC : float
        Phase variable (in radians).
    E_C : float
        Charging energy.
    E_L : float
        Inductive energy.
    c : Qobj
        Coupler annihilation operator.
    c_dag : Qobj
        Coupler creation operator.

    Returns
    -------
    U : Qobj
        The LINC potential energy operator.
    """
    theta = theta_operator(E_C, E_L, c, c_dag)
    U = 2 * M**2 * E_J * np.sin(phi_AC) * (theta / M).cosm()
    return U

In [3]:
# Define system parameters
w0 = 1.0         # qubit energy
A = 0.5          # driving amplitude
w = 1.0          # driving frequency

# Hamiltonian: H = 0.5 * w0 * sigmaz + A * cos(w * t) * sigmax
H0 = 0.5 * w0 * qt.sigmaz()
H1 = A * qt.sigmax()
H = [H0, [H1, 'cos(w * t)']]

# Time-dependent coefficients
args = {'w': w}

# Initial state
psi0 = qt.basis(2, 0)

# Time list
tlist = np.linspace(0, 10, 200)

# Solve using Floquet formalism
floquet_modes, floquet_energies = qt.floquet_modes(H, w, args)
# floquet_modes, floquet_energies = qt.floquet_modes(H, w, args, t=0)
result = qt.floquet_master_equation_solver(H, w, [psi0], tlist, [], args=args)

# Plot the population of state |0>
plt.plot(tlist, np.real(result.expect[0]))
plt.xlabel('Time')
plt.ylabel('Population of |0>')
plt.title('Floquet Simulation: Population of |0>')
plt.show()

c:\Anaconda3\envs\Lib\site-packages\qutip\solver\floquet_bwcomp.py:37: FutureWarning: `floquet_modes` is deprecated. Use `FloquetBasis.mode` instead.
  warnings.warn(FutureWarning(
c:\Anaconda3\envs\Lib\site-packages\qutip\core\coefficient.py:420: UserWarning: `cython`, `setuptools` and `filelock` are required for compilation of string coefficents. Falling back on `eval`.
  warnings.warn(


AttributeError: module 'qutip' has no attribute 'floquet_master_equation_solver'